In [18]:
import pandas as pd
#pd.set_option('future.no_silent_downcasting', True)

In [19]:
#path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA V1.xlsx'
path_xlsx_avance = r'G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [20]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [21]:
# elimina el ultimo registro, para descartar los totales
xlsx_resumen = xlsx_resumen.drop(xlsx_resumen.index[-1])

In [22]:
xlsx_resumen

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
0,1,2025-06-05,12.279029,883.389453,815.18
1,1,2025-06-10,NaN,NaN,1958.91
2,1,2025-06-13,NaN,NaN,774.88
3,1,2025-06-16,NaN,NaN,915.85
4,2,2025-06-10,NaN,NaN,460.89
...,...,...,...,...,...
995,623,2025-06-13,NaN,NaN,118.33
996,623,2025-06-16,NaN,NaN,38.07
997,625,2025-06-10,NaN,NaN,49.04
998,625,2025-06-13,NaN,NaN,156.96


In [23]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador

    df_param['ENTREGAS'] = suma_acumulador
    
    df_param = df_param.dropna(subset=['ENTREGAS'])
    df_param = df_param.drop(columns=['ENTREGADO'])
    #df_param = df_param[df_param['AREA']!=0]
    return df_param

In [24]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [25]:
list_df = []

In [26]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_30868\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_30868\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_30868\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

In [27]:
df_combinado = pd.concat(list_df, ignore_index=True)

In [28]:
df_combinado

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
0,1,2025-06-05,12.279029,883.389453,815.18
1,1,2025-06-16,0.000000,0.000000,3649.64
2,2,2025-06-16,1.819139,46.197223,1783.59
3,3,2025-05-26,1.065976,65.266759,67.71
4,3,2025-06-10,0.939271,57.508982,171.45
...,...,...,...,...,...
529,500,2025-06-16,0.000000,0.000000,5564.42
530,502,2025-06-16,0.000000,0.000000,284.44
531,505,2025-06-16,0.000000,0.000000,157.38
532,508,2025-06-16,0.000000,0.000000,131.04


In [29]:
df_combinado.to_excel(r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')
#df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')